# models

> Fill in a module description here

In [ ]:
#| default_exp models

In [ ]:
#| export
import torch
import torch.nn as nn
import torch.nn.functional as F
from fastai.vision.models.unet import *

In [ ]:
#| export
class TimeSeriesModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(TimeSeriesModel, self).__init__()
        self.conv1 = nn.Conv1d(input_size, 64, kernel_size=7, stride=2)
        self.conv1_bn = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=5, stride=2)
        self.conv2_bn = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3, stride=2)
        self.conv3_bn = nn.BatchNorm1d(256)
        self.conv4 = nn.Conv1d(256, 512, kernel_size=3, stride=2)
        self.conv4_bn = nn.BatchNorm1d(512)
        self.drop = nn.Dropout(0.5)
        self.pool = nn.AdaptiveAvgPool1d(50)
        self.linear = nn.Linear(50*512, output_size)
        self.linear_bn = nn.BatchNorm1d(output_size)
        self.out = nn.Linear(output_size, output_size)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.conv1_bn(x)
        x = F.relu(self.conv2(x))
        x = self.conv2_bn(x)
        x = F.relu(self.conv3(x))
        x = self.conv3_bn(x)
        x = F.relu(self.conv4(x))
        x = self.conv4_bn(x)            
        x = self.pool(x)
        x = x.view(-1, 50*512)
        x = F.relu(self.linear(x))
        x = self.drop(self.linear_bn(x))
        return self.out(x)

In [ ]:
#| export
class StagerNet(nn.Module):
    def __init__(self, channels, dropout_rate=0.5, embed_dim=100):
        super(StagerNet, self).__init__()
        self.conv1 = nn.Conv2d(1, channels, (channels, 1), stride=(1, 1))
        self.conv2 = nn.Conv2d(1, 16, (1, 50), stride=(1, 1))
        self.conv3 = nn.Conv2d(16, 16, (1, 50), stride=(1, 1))
        self.linear1 = nn.Linear(64*channels, embed_dim)
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.batchnorm2 = nn.BatchNorm2d(16)

        self.dropout_rate = dropout_rate
        self.embed_dim = embed_dim
        pass
    
    def forward(self, x):
        # input assumed to be of shape (C,T)=(2,3000)
        x = torch.unsqueeze(x, 1)

        # convolve x with C filters to 1 by T by C
        x = self.conv1(x)
        # permute to (C, T, I)
        x = x.permute(0, 2, 1, 3)

        x = self.conv2(x)
        x = F.relu(F.max_pool2d(x, (1, 13)))
        x = self.batchnorm1(x)
        x = self.conv3(x)
        x = F.relu(F.max_pool2d(x, (1, 13)))
        x = self.batchnorm2(x)

        x = torch.flatten(x, 1) # flatten all but batch dim
        x = F.dropout(x, p=self.dropout_rate)
        x = self.linear1(x)
        return x

In [ ]:
#| export
class StagerUNet_S(nn.Module):
    def __init__(self, channels, dropout_rate=0.5, embed_dim=100, print_shape=False):
        super().__init__()
        self.conv1 = nn.Conv2d(1, channels, (channels, 1), stride=(1, 1), padding=(0, 0))
        self.conv2 = nn.Conv2d(1, 16, (1, 51), stride=(1, 1), padding=(0, 25))
        self.conv3 = nn.Conv2d(16, 16, (1, 51), stride=(1, 1), padding=(0, 25))

        self.dropout_rate = dropout_rate
        self.embed_dim = embed_dim
        
        self.upconv3 = nn.Conv2d(16, 16, (1, 51), stride=(1, 1), padding=(0, 25))
        self.upconv2 = nn.Conv2d(16, 1, (1, 51), stride=(1, 1), padding=(0, 25))
        self.upconv1 = nn.Conv2d(channels, 1, (channels, 1), stride=(1, 1), padding=(0,0))

        self.batchnorm1 = nn.BatchNorm2d(16)
        self.batchnorm2 = nn.BatchNorm2d(16)
        
        self.upbatchnorm1 = nn.BatchNorm2d(16)
        self.upbatchnorm2 = nn.BatchNorm2d(16)
        
        self.print_shape = print_shape
    
    def forward(self, x):

        x = torch.unsqueeze(x, 1)
        
        if self.print_shape: print(x.shape)

        # convolve x with C filters to 1 by T by C
        x = self.conv1(x)
        if self.print_shape: print(x.shape)
        # permute to (C, T, I)
        x = x.permute(0, 2, 1, 3)
        if self.print_shape: print(x.shape)
        x = self.conv2(x)
        
        if self.print_shape: print(x.shape)
        x, ix1 = F.max_pool2d_with_indices(x, (1,5))
        x = F.relu(x)
        if self.print_shape: print(x.shape)
        
        x = self.batchnorm1(x)
        x = self.conv3(x)
        
        if self.print_shape: print(x.shape)
        x, ix2 = F.max_pool2d_with_indices(x, (1,5))
        x = F.relu(x)                               
        x = self.batchnorm2(x)                      
        
        if self.print_shape: print(x.shape)
        x = F.relu(F.max_unpool2d(x, ix2, (1,5)))   
        x = self.upbatchnorm2(x)   
        if self.print_shape: print(x.shape)
        
        x = self.upconv3(x) 
        x = self.upbatchnorm1(x) 
        
        if self.print_shape: print(x.shape)
        
        x = F.relu(F.max_unpool2d(x, ix1, (1,5)))
        x = self.upconv2(x) 
        
        if self.print_shape: print(x.shape)
        
        x = x.permute(0, 2, 1, 3)
        
        if self.print_shape: print(x.shape)
        
        x = torch.squeeze(x, 2)

        
        return x

In [ ]:
#| export
class StagerUNet_M(nn.Module):
    def __init__(self, channels, dropout_rate=0.5, embed_dim=100, print_shape=False):
        super().__init__()
        self.conv1 = nn.Conv2d(1, channels, (channels, 1), stride=(1, 1), padding=(0, 0))
        self.conv2 = nn.Conv2d(1, 16, (1, 51), stride=(1, 1), padding=(0, 25))
        self.conv3 = nn.Conv2d(16, 16, (1, 51), stride=(1, 1), padding=(0, 25))
        self.conv4 = nn.Conv2d(16, 16, (1, 51), stride=(1, 1), padding=(0, 25))

        self.dropout_rate = dropout_rate
        self.embed_dim = embed_dim
        
        self.upconv4 = nn.Conv2d(16, 16, (1, 51), stride=(1, 1), padding=(0, 25))
        self.upconv3 = nn.Conv2d(16, 16, (1, 51), stride=(1, 1), padding=(0, 25))
        self.upconv2 = nn.Conv2d(16, 1, (1, 51), stride=(1, 1), padding=(0, 25))
        self.upconv1 = nn.Conv2d(channels, 1, (channels, 1), stride=(1, 1), padding=(0,0))

        self.batchnorm1 = nn.BatchNorm2d(16)
        self.batchnorm2 = nn.BatchNorm2d(16)
        self.batchnorm3 = nn.BatchNorm2d(16)
        
        self.upbatchnorm1 = nn.BatchNorm2d(16)
        self.upbatchnorm2 = nn.BatchNorm2d(16)
        self.upbatchnorm3 = nn.BatchNorm2d(16)
        
        self.print_shape = print_shape

    
    def forward(self, x):

        x = torch.unsqueeze(x, 1)
        
        if self.print_shape: print(x.shape)


        x = self.conv1(x)
        if self.print_shape: print(x.shape)

        x = x.permute(0, 2, 1, 3)
        if self.print_shape: print(x.shape)
        x = self.conv2(x)
        
        if self.print_shape: print(x.shape)
        x, ix1 = F.max_pool2d_with_indices(x, (1,5))
        x = F.relu(x)
        if self.print_shape: print(x.shape)
        x = self.batchnorm1(x)
        
        x = self.conv3(x)
        
        if self.print_shape: print(x.shape)
        x, ix2 = F.max_pool2d_with_indices(x, (1,5))
        x = F.relu(x)                               
        x = self.batchnorm2(x)     
        
        x = self.conv4(x)
        
        if self.print_shape: print(x.shape)
        x, ix3 = F.max_pool2d_with_indices(x, (1,5))
        x = F.relu(x)                               
        x = self.batchnorm3(x) 
        
        
        if self.print_shape: print(x.shape)
        x = F.relu(F.max_unpool2d(x, ix3, (1,5)))   
        x = self.upbatchnorm3(x)   
        if self.print_shape: print(x.shape)
        x = self.upconv4(x) 
        
        
        if self.print_shape: print(x.shape)
        x = F.relu(F.max_unpool2d(x, ix2, (1,5)))   
        x = self.upbatchnorm2(x)   
        if self.print_shape: print(x.shape)
        
        x = self.upconv3(x) 
        
        
        if self.print_shape: print(x.shape)
        
        x = F.relu(F.max_unpool2d(x, ix1, (1,5)))
        x = self.upbatchnorm1(x) 
        
        x = self.upconv2(x) 
        if self.print_shape: print(x.shape)
        
        x = x.permute(0, 2, 1, 3)
        
        if self.print_shape: print(x.shape)
        
        #x = self.conv1(x)
        
        x = torch.squeeze(x, 2)

        
        return x

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()